In [3]:
import cv2

In [4]:
# Load object to artist ID's mapping.
import json
o2a = open('oid_to_artist_id.json')
object_to_artists = json.load(o2a)

In [5]:
from utils.data_utils import *

In [6]:
def compute_hog(image):
    hog = cv2.HOGDescriptor()
    image = np.transpose(image, [1,2,0])
    hist = hog.compute(image)
    return hist

In [21]:

params = cv2.SimpleBlobDetector_Params()
detector = cv2.SimpleBlobDetector_create(params)
def compute_keypoints(image): 
    # Detect blobs.
#     print(image.shape)
    gray_image = np.mean(image, axis=0).astype('uint8')
    keypoints = detector.detect(gray_image)
    keypoints = np.array(keypoints)
    print (keypoints.shape)
    return keypoints


In [8]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss


In [18]:

def run_epoch(data_path, val_path, model, feature_extractor, object_to_artists, classes, loss):
    batcherator = batch_generator(data_path)
    
#     tl_batcherator = batch_generator(data_path)
#     vl_batcherator = batch_generator(val_path)
    
    train_losses = []
    val_losses = []
    
    batch_count = 0
    
    def get_xy(X_chunk, y_chunk):
        X_feat = []
        for i in np.arange(X_chunk.shape[0]):    
            X_feat.append(feature_extractor(X_chunk[i].astype('uint8')))
        X_feat = np.array(X_feat)
        print (X_feat.shape)
        # flatten
        X_feat = np.reshape(X_feat, (X_feat.shape[0], -1))
        return X_feat, y_chunk
    # Train model
    try:
        while True:
            current_batch = next(batcherator)
            X_chunk, y_chunk = get_array_from_batch(data_path, current_batch, object_to_artists)
            X_feat, y_chunk = get_xy(X_chunk, y_chunk)
            model.partial_fit(X_feat, y_chunk, classes=classes)
            
            batch_count += 1
            if batch_count % 10 == 0:
                print ('Batch count: ' + str(batch_count))
#                 train_batch = next(tl_batcherator)
#                 val_batch = next(vl_batcherator)
                
#                 X_chunk_train, y_chunk_train = get_array_from_batch(data_path, train_batch, object_to_artists)
#                 X_chunk_val, y_chunk_val = get_array_from_batch(val_path, val_batch, object_to_artists)
#                 X_feat_train, y_chunk_train = get_xy(X_chunk_train, y_chunk_train)
#                 X_feat_val, y_chunk_val = get_xy(X_chunk_val, y_chunk_val)
                
                
#                 tl = loss(y_chunk_train, X_feat_train, model, classes)
#                 vl = loss(y_chunk_val, X_feat_val, model, classes)
                
#                 train_losses.append(tl)
#                 val_losses.append(vl)
#                 print ('Training loss: ' + str(tl))
#                 print ('Val loss: ' + str(vl))
#                 print ('\n')
                
#             if batch_count > 10:
#                 raise(StopIteration)
    except StopIteration:
        pass
        
    return model, train_losses, val_losses
 
#     Now make predictions with trained model
#     y_predicted = model.predict(X_test)

In [10]:
def compute_accuracy(y_pred, y_actual):
    return float(np.sum(y_pred == y_actual)) / y_pred.shape[0]

In [ ]:
def predict(data_path, model, feature_extractor, object_to_artists, max_batches=None):
    batch_counter = 0
    accuracy_sum = 0
    batcherator = batch_generator(data_path)
    
    try:
        while True:
            current_batch = next(batcherator)
            X_chunk, y_chunk = get_array_from_batch(data_path, current_batch, object_to_artists)
            X_feat = []
            for i in np.arange(X_chunk.shape[0]):    
                X_feat.append(feature_extractor(X_chunk[i].astype('uint8')))

            X_feat = np.array(X_feat)
            
            # flatten
            X_feat = np.reshape(X_feat, (X_feat.shape[0], -1))
            y_predicted = model.predict(X_feat)
            accuracy = compute_accuracy(y_predicted, y_chunk)
           
            accuracy_sum += accuracy
            batch_counter += 1
            
            if (max_batches is not None) and (batch_counter > max_batches):
                raise(StopIteration)
    except StopIteration:
        pass
    
    print ('Accuracy for ' + data_path + ": " + str(float(accuracy_sum) / batch_counter))
    return accuracy
        

In [12]:
# def log_loss(Y, X, model, classes)    
#     probas = model.predict_proba(X)
#     assert_array_almost_equal(np.sum(probas, axis=1), np.ones(len(X_test)))

#     # Check that log-loss of calibrated classifier is smaller than
#     # log-loss of naively turned OvR decision function to probabilities
#     # via softmax
#     def softmax(y_pred):
#         e = np.exp(-y_pred)
#         return e / e.sum(axis=1).reshape(-1, 1)
#     uncalibrated_log_loss = \
#         log_loss(Y, softmax(clf.decision_function(X)))
#     calibrated_log_loss = log_loss(Y, probas)

In [16]:
# clf = fit_svm(object_to_artists, compute_hog, None)
# predict(clf, compute_hog, 'data/train', None)
# predict(clf, compute_hog, 'data/test', None)
model = SGDClassifier(shuffle=True, loss='log')
classes = np.arange(25)
train_accuracies = []
val_accuracies = []
for i in np.arange(20):
    print('Epoch #' + str(i))
    model, train_losses_curr, val_losses_curr = run_epoch('data/train', 'data/val', model, \
                                                compute_hog, object_to_artists, classes, log_loss)
    train_accuracies.append(predict('data/train', model, compute_hog, object_to_artists, 10))
    val_accuracies.append(predict('data/test', model, compute_hog, object_to_artists, None))


Epoch #0
Batch count: 10


KeyboardInterrupt: 

In [22]:
sift_model = SGDClassifier(shuffle=True, loss='log')
classes = np.arange(25)

for i in np.arange(20):
    print('Epoch #' + str(i))
    sift_model, train_losses_curr, val_losses_curr = run_epoch('data/train', 'data/val', sift_model, \
                                                compute_keypoints, object_to_artists, classes, log_loss)
    predict('data/train', sift_model, compute_keypoints, object_to_artists, 10)
    predict('data/test', sift_model, compute_keypoints, object_to_artists, None)

Epoch #0
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(2,)
(0,)
(1,)
(1,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(2,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(2,)
(0,)
(0,)
(0,)
(1,)
(3,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(1,)
(1,)
(0,)
(50,)


ValueError: setting an array element with a sequence.